# Wisdm Reddit Code Review 
- pull data from reddit using Sid's code
- pull data from reddit using wisdm code
- compare and get feedback 
- 

## Sid's Code

In [3]:
#!pip install praw
import praw
import pandas as pd

In [25]:
# Creates Reddit user instance 
# Connects to Reddit using Wisdm account

reddit = praw.Reddit(
    client_id="auD_kIwyQ1r3hfxTQEYuGw",
    client_secret="XyLGeMB1mJqoaB0lCXrk4Jtmy515AA",
    password="wisdmai1234",
    user_agent="wisdm",
    username="Dramatic-Ad-9651",
    check_for_async=False
)

In [5]:
# Confirm user name is Dramatic-Ad-9651
print(reddit.user.me())

Dramatic-Ad-9651


In [6]:
# specifies what sub reddit we are looking in
subreddit = reddit.subreddit('wallstreetbets')


# Tells the call to look at the 'hot' posts in the specified subreddit 
hot = subreddit.hot(limit = 15)

In [122]:
# This is will be provided by the scoping algorithm

tickers = list(set(['SPY','TSLA','PYPL', 'GME', 'VIX', 'AMD', 'F', 'BBBY', 'NFLX', 'NVDA', 'AAPL', 'INTC', 'FSR', 'QQQ', 'TLRY', 'MSFT', 'TWTR', 'SNAP', 'HOOD', 'WMT', 'PTON', 'WISH', 'CPRX', 'AMC', 'SNDL', 'AMZN', 'DIS', 'NIO', 'FB', 'NFLX', 'LCID', 'NVDA']))

tickers[0:5]

['AAPL', 'MSFT', 'F', 'TSLA', 'NVDA']

In [123]:
def pull_Reddit():
    posts = []
    #subreddit = reddit.subreddit('wallstreetbets')
    subreddits = [ 'wallstreetbets', 'stocks',  'investing', 'finance']
    for thread in subreddits: 
        subreddit = reddit.subreddit(thread)
        for post in subreddit.hot(limit=1000000):
            checker = False
            for word in post.title.split():
                if word in tickers:
                    checker = True
            for word in post.selftext.split():
                if word in tickers:
                    checker = True
            if checker:
                posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
            
    posts_df = pd.DataFrame(posts, columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
    return posts_df


#### Feedback
* might be helpful to include print statements to tell us we are in the process 
* make subreddits a parameter 
* make stock tickers a parameter 

In [124]:
%%timeit 
pull_Reddit() 

27 s ± 1.17 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


# 

In [125]:
df = pull_Reddit() 
df.head()

,title,score,id,subreddit,url,num_comments,body,created
0,by the power of QQQ puts I can afford home own...,130,ykkit5,wallstreetbets,https://i.redd.it/3136fea7amx91.jpg,2,,1.667429e+09
1,SPY Put Pivot Tendies: $400 to $8k and holding,114,ykj8wg,wallstreetbets,https://i.redd.it/25w0kfln2mx91.jpg,34,,1.667427e+09
2,congrats to those who correctly timed SPY opti...,140,ykeket,wallstreetbets,https://i.redd.it/oh56h9p3smx91.jpg,17,,1.667417e+09
3,"$14,000 day gain thanks to SPY puts",56,ykfl21,wallstreetbets,https://www.reddit.com/gallery/ykfl21,17,,1.667419e+09
4,"11-2-22 SPY/ ES futures, Apple and Tesla Daily TA",23,ykh7d5,wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,32,And just like that the bullish momentum and st...,1.667423e+09


In [126]:
df.shape

(111, 8)

# Wisdm Archive Reddit Code 

In [67]:
import pandas as pd
from datetime import datetime
from praw.models import MoreComments
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
reddit = praw.Reddit(
    client_id="auD_kIwyQ1r3hfxTQEYuGw",
    client_secret="XyLGeMB1mJqoaB0lCXrk4Jtmy515AA",
    password="wisdmai1234",
    user_agent="wisdm",
    username="Dramatic-Ad-9651",
    check_for_async=False
)

In [168]:
def pull_reddit_v2(): 
  #stocklist = ('gme', 'amc', 'tsla', 'hood', 'fb', 'twtr', 'spy', 'webr', 'evtl', 'arqq', 'bbby', 'tsm', 'ppi', 'nvda')
  stocklist = ('SPY','TSLA','PYPL', 'GME', 'VIX', 'AMD', 'F', 'BBBY', 'NFLX', 'NVDA', 'AAPL', 'INTC', 'FSR', 'QQQ', 'TLRY', 'MSFT', 'TWTR', 'SNAP', 'HOOD', 'WMT', 'PTON', 'WISH', 'CPRX', 'AMC', 'SNDL', 'AMZN', 'DIS', 'NIO', 'FB', 'NFLX', 'LCID', 'NVDA')
  subreddits = ('wallstreetbets', 'stocks',  'investing', 'finance')
  posts = pd.DataFrame()
  for stock in stocklist:
    #print(stock)
    for sub in subreddits:
      #print(sub)
      subreddit = reddit.subreddit(sub)
      for post in subreddit.search(stock, sort = 'new', time_filter = "week",limit=None):
        if stock.lower() not in post.title.lower():
          #print(stock) 
          #print(post.title.lower())
          continue
        pslice = pd.DataFrame()
        pslice = pslice.append({
            'stock': stock,
            'subreddit': post.subreddit,
            'type': 'POST',
            'post_title': post.title,
            'selftext': post.selftext,
            #'upvote_ratio': post.upvote_ratio,
            #'ups': post.ups,
            'ups': post.score,
            #'num_comments': post.num_comments,
            'id': post.id,
            'time': datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%dT%H:%M:%SZ')
          }, ignore_index=True)
        #if post.id in posts['id']:
          #continue
        posts = posts.append(pslice, ignore_index=True)
        #post.comments.replace_more(limit=None)
        cslice = pd.DataFrame()
        for com in post.comments:
          if isinstance(com, MoreComments):
            continue
          cslice = cslice.append({
              'stock': stock,
              'subreddit': post.subreddit,
              'type': 'COMMENT',
              'post_title': post.title,
              'selftext': com.body,
              #'upvote_ratio': com.upvote_ratio,
              #'ups': com.ups,
              'ups': com.score,
              #'num_comments': post.num_comments,
              'id': post.id,
              'time': datetime.fromtimestamp(com.created_utc).strftime('%Y-%m-%dT%H:%M:%SZ'),
            }, ignore_index=True)
        posts = posts.append(cslice, ignore_index=True)

  return posts

In [170]:
%%timeit 
pull_reddit_v2()

5min ± 907 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [171]:
df_2 = pull_reddit_v2() 
df_2.head()

,stock,subreddit,type,post_title,selftext,ups,id,time
0,SPY,wallstreetbets,POST,SPY Put Pivot Tendies: $400 to $8k and holding,,132,ykj8wg,2022-11-02T15:07:57Z
1,SPY,wallstreetbets,COMMENT,SPY Put Pivot Tendies: $400 to $8k and holding,\n**User Report**| | | |\n:--|:--|:--|:--\n**T...,1,ykj8wg,2022-11-02T15:08:02Z
2,SPY,wallstreetbets,COMMENT,SPY Put Pivot Tendies: $400 to $8k and holding,"I'm confused, how did you buy 7 of those optio...",21,ykj8wg,2022-11-02T15:24:16Z
3,SPY,wallstreetbets,COMMENT,SPY Put Pivot Tendies: $400 to $8k and holding,Congrats,10,ykj8wg,2022-11-02T15:12:39Z
4,SPY,wallstreetbets,COMMENT,SPY Put Pivot Tendies: $400 to $8k and holding,Started with $20 a month ago. Now at $120. I w...,6,ykj8wg,2022-11-02T16:21:35Z


In [172]:
df_2.shape

(5118, 8)

# EDA

In [162]:
df_v1 = df.copy() 
df_v2 = df_2.copy()

In [163]:
df_v1.head()

,title,score,id,subreddit,url,num_comments,body,created
0,by the power of QQQ puts I can afford home own...,130,ykkit5,wallstreetbets,https://i.redd.it/3136fea7amx91.jpg,2,,1.667429e+09
1,SPY Put Pivot Tendies: $400 to $8k and holding,114,ykj8wg,wallstreetbets,https://i.redd.it/25w0kfln2mx91.jpg,34,,1.667427e+09
2,congrats to those who correctly timed SPY opti...,140,ykeket,wallstreetbets,https://i.redd.it/oh56h9p3smx91.jpg,17,,1.667417e+09
3,"$14,000 day gain thanks to SPY puts",56,ykfl21,wallstreetbets,https://www.reddit.com/gallery/ykfl21,17,,1.667419e+09
4,"11-2-22 SPY/ ES futures, Apple and Tesla Daily TA",23,ykh7d5,wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,32,And just like that the bullish momentum and st...,1.667423e+09


In [164]:
df_v2.head()

,stock,subreddit,type,post_title,selftext,ups,id,time
0,SPY,wallstreetbets,POST,SPY Put Pivot Tendies: $400 to $8k and holding,,115,ykj8wg,2022-11-02T15:07:57Z
1,SPY,wallstreetbets,COMMENT,SPY Put Pivot Tendies: $400 to $8k and holding,\n**User Report**| | | |\n:--|:--|:--|:--\n**T...,1,ykj8wg,2022-11-02T15:08:02Z
2,SPY,wallstreetbets,COMMENT,SPY Put Pivot Tendies: $400 to $8k and holding,"I'm confused, how did you buy 7 of those optio...",17,ykj8wg,2022-11-02T15:24:16Z
3,SPY,wallstreetbets,COMMENT,SPY Put Pivot Tendies: $400 to $8k and holding,Congrats,9,ykj8wg,2022-11-02T15:12:39Z
4,SPY,wallstreetbets,COMMENT,SPY Put Pivot Tendies: $400 to $8k and holding,Started with $20 a month ago. Now at $120. I w...,6,ykj8wg,2022-11-02T16:21:35Z


In [165]:
df_v1.shape, df_v2.shape

((111, 8), (5110, 8))

* explore score, upvotes, comments

- first function does not pull enough content 
- first does not include comments only post titles 
- second function might be pulling in too much noise 
- second function might be pulling too many comments